# The Solow Model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import sympy as sm
import ipywidgets as widgets
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# The general solow model - definition

Consider the **standard Solow-model** where:

1. $K_t$ is capital
2. $L_t$ is labor (growing with a constant rate of $n$)
3. $A_t$ is technology (growing with a constant rate of $g$)
4. $Y_t = F(K_t,A_tL_t)$ is GDP

The Cobb Douglas function for the representative firm is given by:
$$
Y_t=K_t^\alpha (A_t L_t)^{1-\alpha}
$$

**Saving** is a constant fraction of GDP

$$ 
S_t = sY_t,\,s\in(0,1)
$$

such that **capital accumulates** according to

$$
K_{t+1}=S_{t}+(1-\delta)K_{t}=sF(K_{t},A_{t}L_{t})+(1-\delta)K_{t}, \delta \in (0,1)
$$

The **production function** has **constant-return to scale** such that

$$
\frac{Y_{t}}{A_{t}L_{t}}=\frac{F(K_{t},A_{t}L_{t})}{A_{t}L_{t}}=F(\tilde{k}_{t},1)\equiv f(\tilde{k}_{t})
$$

where $\tilde{k}_t = \frac{K_t}{A_{t}L_{t}}$ is the technology adjusted capital-labor ratio.

The **transition equation** then becomes

$$
\tilde{k}_{t+1}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}_{t})+(1-\delta)\tilde{k}_{t}]
$$

If the **production function** is **Cobb-Douglas** then

$$
F(K_{t},A_{t}L_{t})=K_{t}^{\alpha}(A_{t}L_{t})^{1-\alpha}\Rightarrow f(\tilde{k}_{t})=\tilde{k}_{t}^{\alpha}
$$

If it is **CES** (with $\beta < 1, \beta \neq 0$) then

$$
F(K_{t},A_{t}L_{t})=(\alpha K_{t}^{\beta}+(1-\alpha)(A_{t}L_{t})^{\beta})^{\frac{1}{\beta}}\Rightarrow f(\tilde{k}_{t})=(\alpha\tilde{k}_{t}^{\beta}+(1-\alpha))^{\frac{1}{\beta}}
$$


when using sympy to fine the steady state, we have to solve the following:

$$ \tilde{k}^{\ast}= \frac{1}{(1+n)(1+g)}[sf(\tilde{k}^{\ast})+(1-\delta)\tilde{k}^{\ast}] $$


In [2]:
# use sympy for defining symbols and functions

# define symbols
k = sm.symbols('k')
s = sm.symbols('s')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
n = sm.symbols('n')
g = sm.symbols('g')

# define function
f = sm.Function('f')(k)
f = k**alpha

# Define function to solve for steady state
ss = sm.Eq(k,(s*f + (1-delta)*k )/((1+n)*(1+g)))


The steady state can now be solved analytically

In [3]:
# solving for steady state analytically
ss_sol = sm.solve(ss,k)[0]
ss_sol


((delta + g*n + g + n)/s)**(1/(alpha - 1))

In [4]:
# Transformation into python function
ss_sol_func = sm.lambdify((s,alpha,delta,n,g),ss_sol)


We can now set parameter values and calculate the steady state

In [5]:
# Define parameters
s = 0.3
alpha = 0.3
delta = 0.1
n = 0.01
g = 0.02

In [6]:
# show the result of the steady state calculated analytically

print('The steady state is: ',ss_sol_func(s,alpha,delta,n,g))

The steady state is:  3.2951243994886896


# Calculation numerically

In stead of finding the steady state analytically, one can also do it numerically. This is done by creating a root finding problem, and then solve for the root

$$
0 = \frac{1}{(1+n)(1+g)}[sf(\tilde{k}^{\ast})+(1-\delta)\tilde{k}^{\ast}] - \tilde{k}^{\ast} 
$$

In [7]:
# Define parameters
s = 0.3
alpha = 0.3
delta = 0.1
n = 0.01
g = 0.02

# write your code here
f = lambda k: k**alpha
obj_kss = lambda kss: kss - (s*f(kss) + (1-delta)*kss)/((1+g)*(1+n))
result = optimize.root_scalar(obj_kss,bracket=[0.1,100],method='brentq')

print('the steady state for k is', result.root)  

the steady state for k is 3.295124399488689


# Visualising the Solow model

In [3]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def cobb_douglas(K, L, A, alpha):
    return A * K**alpha * L**(1 - alpha)

def plot_solow_model(s, alpha, delta, n, g, T):
    K0 = 1
    L0 = 1
    A0 = 1
    Y0 = 1
    Kt = 1

    K = [K0]
    L = [L0]
    A = [A0]
    Y = [Y0]
    Kt = [Kt]

    for t in range(T):
        Kt_new = s * cobb_douglas(K[t] / (A[t] * L[t]), 1, 1, alpha) + (1 - delta) * K[t] / (A[t] * L[t])
        K_new = Kt_new * A[t] * L[t]
        K.append(K_new)

        L_new = (1 + n) * L[t]
        L.append(L_new)

        A_new = (1 + g) * A[t]
        A.append(A_new)

        Y_new = cobb_douglas(K_new, L_new, A_new, alpha)
        Y.append(Y_new)

        Kt.append(Kt_new)

    plt.plot(Kt)
    plt.title('Plot of Kt')
    plt.xlabel('Time')
    plt.ylabel('Kt')
    plt.show()

widgets.interact(
    plot_solow_model,
    s=widgets.FloatSlider(value=0.3, min=0, max=1, step=0.01, description='s'),
    alpha=widgets.FloatSlider(value=0.3, min=0, max=1, step=0.01, description='alpha'),
    delta=widgets.FloatSlider(value=0.1, min=0, max=1, step=0.01, description='delta'),
    n=widgets.FloatSlider(value=0.01, min=0, max=0.1, step=0.001, description='n'),
    g=widgets.FloatSlider(value=0.02, min=0, max=0.1, step=0.001, description='g'),
    T=widgets.IntSlider(value=100, min=10, max=1000, step=10, description='T')
)


interactive(children=(FloatSlider(value=0.3, description='s', max=1.0, step=0.01), FloatSlider(value=0.3, desc…

<function __main__.plot_solow_model(s, alpha, delta, n, g, T)>

## Solow model with land and oil

We add oil and land to the model to add more dimensions to the model

Now the production function for the representative firm is given by:

$$ 
Y_t=K_t^\alpha (A_t L_t)^\beta X^\kappa E_t^\epsilon, \hspace{5mm}    \alpha>0, \beta>0, \kappa>0, \epsilon>0, \hspace{5mm}  \alpha+\beta+\kappa+\epsilon=1
$$

Where $X$ is land and $E_t$ is oil

$$
L_t=(1+n)L_t, \hspace{5mm}   n\geq0
$$

$$
A_{t+1}=(1+g)A_t, \hspace{5mm} g\geq0
$$

The depletion of the natural ressource is described by: 
$$
R_{t+1}=R_t-E_t
$$

Where $R_t$ is the remaining stock of oil in each period and $E_t$ is how much is oil is consumed.

the oil consumption is given as:

$$
E_t=s_E R_t , \hspace{5mm} 0<s_E<1
$$

**Saving** is still a constant fraction of GDP

$$ 
S_t = sY_t,\,s\in(0,1)
$$

such that **capital accumulates** according to

$$
K_{t+1}=S_{t}+(1-\delta)K_{t}=sF(K_{t},A_{t}L_{t},X,E_t)+(1-\delta)K_{t}, \delta \in (0,1)
$$

Inserting $E_t=s_E R_t$ into the production function and dividing both sides by $L_t$ yields

$$
y_t=k_t^\alpha A_t^\beta (\frac{X}{L_t})^\kappa (\frac{s_E R_t}{L_t})^\epsilon = s_E^\epsilon X^\kappa k_t^\alpha A_t^\beta R_t^\epsilon L_t^{-\kappa-\epsilon}
$$

By taking logs we get the following:

$$
g_t^y=\alpha g_t^k+\beta g_t^A + \epsilon g_t^R - (\kappa+\epsilon)g_t^L \approx \alpha g_t^k +\beta g - \epsilon s_E - (\kappa+\epsilon)n
$$

As $g_t^y$ and $g_t^k$ converge towards the same rate they are set equal. This yields:

$$
g^y=\frac{\beta}{\beta+\kappa+\epsilon}g-\frac{\kappa + \epsilon}{\beta+\kappa+\epsilon}n - \frac{\epsilon}{\beta+\kappa+\epsilon}s_E
$$

# Visualising the difference between the models

In [12]:
#Define oil function

# Define cobb douglas function with capital, labor, technology, oil and land

import matplotlib.pyplot as plt
import ipywidgets as widgets

def solow_model_with_land_and_oil(K, A, L, X, Et, alpha, beta, kappa, epsilon):
    return K**alpha * (A * L)**beta * X**kappa * Et**epsilon

def plot_solow_model_with_land_and_oil(T, s, alpha, beta, kappa, n, g, s_E, delta):
    epsilon = 1 - alpha - beta - kappa
    # Initial values
    K0 = 1
    L0 = 1
    A0 = 1
    Y0 = 1
    R0 = 100
    X = 1

    K = [K0]
    L = [L0]
    A = [A0]
    Y = [Y0]
    R = [R0]
    E = [s_E * R0]

    # Time loop
    for t in range(T):
        L_new = (1 + n) * L[t]
        L.append(L_new)

        A_new = (1 + g) * A[t]
        A.append(A_new)

        Et_new = s_E * R[t]
        E.append(Et_new)

        Rt_new = R[t] - Et_new
        R.append(Rt_new)

        Y_new = solow_model_with_land_and_oil(K[t], A_new, L_new, X, Et_new, alpha, beta, kappa, epsilon)
        Y.append(Y_new)

        K_new = s * Y_new + (1 - delta) * K[t]
        K.append(K_new)

    plt.plot(K)
    plt.title('Plot of Kt')
    plt.xlabel('Time')
    plt.ylabel('Kt')
    plt.show()

alpha_slider = widgets.FloatSlider(value=0.3, min=0, max=1, step=0.01, description='alpha')
beta_slider = widgets.FloatSlider(value=0.3, min=0, max=1, step=0.01, description='beta')
kappa_slider = widgets.FloatSlider(value=0.1, min=0, max=1, step=0.01, description='kappa')

def on_alpha_slider_change(change):
    new_alpha = change.new
    new_beta = min(beta_slider.value, 1 - new_alpha - kappa_slider.value)
    beta_slider.value = new_beta
    kappa_slider.max = 1 - new_alpha - new_beta

def on_beta_slider_change(change):
    new_beta = change.new
    new_alpha = min(alpha_slider.value, 1 - new_beta - kappa_slider.value)
    alpha_slider.value = new_alpha
    kappa_slider.max = 1 - new_alpha - new_beta

def on_kappa_slider_change(change):
    new_kappa = change.new
    new_alpha = min(alpha_slider.value, 1 - new_kappa - beta_slider.value)
    alpha_slider.value = new_alpha
    beta_slider.max = 1 - new_alpha - new_kappa

alpha_slider.observe(on_alpha_slider_change, names='value')
beta_slider.observe(on_beta_slider_change, names='value')
kappa_slider.observe(on_kappa_slider_change, names='value')

widgets.interact(
    plot_solow_model_with_land_and_oil,
    T=widgets.IntSlider(value=100, min=10, max=1000, step=10, description='T'),
    s=widgets.FloatSlider(value=0.3, min=0, max=1, step=0.01, description='s'),
    alpha=alpha_slider,
    beta=beta_slider,
    kappa=kappa_slider,
    n=widgets.FloatSlider(value=0.01, min=0, max=0.1, step=0.001, description='n'),
    g=widgets.FloatSlider(value=0.02, min=0, max=0.1, step=0.001, description='g'),
    s_E=widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='s_E'),
    delta=widgets.FloatSlider(value=0.1, min=0, max=1, step=0.01, description='delta')
)


interactive(children=(IntSlider(value=100, description='T', max=1000, min=10, step=10), FloatSlider(value=0.3,…

<function __main__.plot_solow_model_with_land_and_oil(T, s, alpha, beta, kappa, n, g, s_E, delta)>